In [1]:
import pandas as pd
from gurobipy import *

In [2]:
problem2 = pd.read_excel('OR2_06_quiz_data .xlsx', 'Problem 2', header=[0,1])

In [3]:
problem2.head()

District \n(from) District (to)                                Population  \
  Unnamed: 0_level_1             1  2  3  4  5  6   7   8 Unnamed: 9_level_1   
0                  1             0  3  4  6  8  9   8  10                 40   
1                  2             3  0  5  4  8  6  12   9                 30   
2                  3             4  5  0  2  2  3   5   7                 35   
3                  4             6  4  2  0  3  2   5   4                 20   
4                  5             8  8  2  3  0  2   2   4                 15   

   ...                                                                  
   ...    &   1 &   2 &   3 &   4 &   5 &   6 &    7 &    8 &      \\   
0  ...  1 &   0 &   3 &   4 &   6 &   8 &   9 &    8 &   10 &   40 \\   
1  ...  2 &   3 &   0 &   5 &   4 &   8 &   6 &   12 &    9 &   30 \\   
2  ...  3 &   4 &   5 &   0 &   2 &   2 &   3 &    5 &    7 &   35 \\   
3  ...  4 &   6 &   4 &   2 &   0 &   3 &   2 &    5 &    4 &   20 \\   
4  ...  5 &   8 &   8 &   2 &   3 &   0 &   2 &    2 &    4 &   15 \\   

[5 rows x 21 columns]

In [4]:
problem2.drop(columns=problem2.columns[-11:], inplace=True)

In [5]:
problem2.head()

District \n(from) District (to)                                Population
  Unnamed: 0_level_1             1  2  3  4  5  6   7   8 Unnamed: 9_level_1
0                  1             0  3  4  6  8  9   8  10                 40
1                  2             3  0  5  4  8  6  12   9                 30
2                  3             4  5  0  2  2  3   5   7                 35
3                  4             6  4  2  0  3  2   5   4                 20
4                  5             8  8  2  3  0  2   2   4                 15

In [6]:
problem2.columns = ['District (from)', 0,1,2,3,4,5,6,7,'Population']
districts = problem2['District (from)']
distance = problem2.iloc[:, 1:9]
population = problem2['Population']

In [7]:
problem2.head()

,District (from),0,1,2,3,4,5,6,7,Population
0,1,0,3,4,6,8,9,8,10,40
1,2,3,0,5,4,8,6,12,9,30
2,3,4,5,0,2,2,3,5,7,35
3,4,6,4,2,0,3,2,5,4,20
4,5,8,8,2,3,0,2,2,4,15


In [8]:
model_2 = Model("q2")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-10


In [9]:
y={}
w={}
x={}

for i in districts:
    y[i] = model_2.addVar(lb = 0, vtype = GRB.BINARY, name = "y" + str(i))
    w[i] = model_2.addVar(lb = 0, vtype = GRB.INTEGER, name = "w" + str(i))
    for j in districts:
        x[i,j] = model_2.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(i) + str(j))

In [10]:
firefighting_times = model_2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "firefighting_times")

In [11]:
model_2.setObjective(firefighting_times, GRB.MINIMIZE)

In [12]:
#add constraints and name them
model_2.addConstrs((quicksum(x[i,j] for i in districts) == 1 for j in districts))
model_2.addConstrs((x[i,j] <= y[i] for i in districts for j in districts))
model_2.addConstr((quicksum(y[i] for i in districts) <= 2))
model_2.addConstrs((w[i] >= quicksum(x[j, i] * distance[j-1][i-1]  for j in districts) for i in districts))
model_2.addConstrs((firefighting_times >= w[i] * population[i-1] for i in districts))

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>}

In [13]:
model_2.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 89 rows, 81 columns and 280 nonzeros
Model fingerprint: 0x4603837a
Variable types: 1 continuous, 80 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 240.0000000
Presolve removed 22 rows and 22 columns
Presolve time: 0.03s
Presolved: 67 rows, 59 columns, 208 nonzeros
Variable types: 0 continuous, 59 integer (58 binary)

Root relaxation: objective 8.289474e+01, 52 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   82.89474    0  

In [15]:
#writing the model to a file
model_2.write("q2.lp")

In [16]:
#writing the solution to a file
model_2.write("q2.sol")